In [1]:
!python -m pip install pyyaml==5.1 #pyYAMl for configuratio and detectron 2 requires <= 5.1 version
import sys #access to python runtime environment
import os #file and  directory operations
import distutils.core  #used to build and install python modules from setup.py


!git clone 'https://github.com/facebookresearch/detectron2' #clones into current directory


dist = distutils.core.run_setup("./detectron2/setup.py") #programmtically runs the setup.py script inside the cloned folder

!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
#install all the dependencies under install requires from the setip.py

sys.path.insert(0, os.path.abspath('./detectron2'))
#adds the absolute path of ./detectron2 to the beginning of sys path

The system cannot find the file specified.
'git' is not recognized as an internal or external command,
operable program or batch file.


FileNotFoundError: [Errno 2] No such file or directory: './detectron2/setup.py'

In [ ]:
import torch
import detectron2
torch.__version__


In [ ]:
detectron2.__version__

In [ ]:
import detectron2  # import detectron 2 pakage
from detectron2.utils.logger import setup_logger  # loging system of detectron
setup_logger()  #initilization of logger



# some common library

import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow


# detectron 2 utilities

from detectron2 import model_zoo  # pre train model downloding krne ke liye
from detectron2.engine import DefaultPredictor  #wrapper around detectron 2 inference pipeline
from detectron2.config import get_cfg   # it will give you path and weight configuration artitechure
from detectron2.utils.visualizer import Visualizer  # to visulized the boxes
from detectron2.data import MetadataCatalog, DatasetCatalog   # metadatacatlog >> store metadata for dataset (class_name,ket_point)

# datasetcatlog >> register and retrive the datasets for detectron2

In [ ]:
import requests
import os

def download_image(url, filename):
  headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }
  response = requests.get(url, stream=True, headers = headers)
  response.raise_for_status()

  with open(filename, "wb") as out_file:
    for chunk in response.iter_content(chunk_size=8192):
      out_file.write(chunk)


  print(f"Image downloaded successfully to {filename}")

download_image("https://images.pexels.com/photos/13872248/pexels-photo-13872248.jpeg", "pexels-photo-13872248.jpg")

In [ ]:
# import requests
# headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
#     }
# response = requests.get("https://images.pexels.com/photos/13872248/pexels-photo-13872248.jpeg", stream=True, headers = headers)
# response.raise_for_status()

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

im = cv2.imread("/content/pexels-photo-13872248.jpg")
cv2_imshow(im)

In [ ]:
# pre train model use krenge



cfg = get_cfg() #it initializes a fresh detectron2 configuration object, it holds all model weights and config

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
#during inference the model will only keep predicted boxes with a confidence score>=0.5
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml")
#retrieves the pretrained weights for the specificed moder
cfg.MODEL.DEVICE = "cpu" #cuda for gpu
predictor = DefaultPredictor(cfg) #builds the model using cfg
outputs = predictor(im)

In [ ]:
outputs

In [ ]:
outputs["instances"].pred_classes

In [ ]:
outputs["instances"].pred_boxes

In [ ]:
# model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")

In [ ]:
#visualisation

v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])